# Power of Graph Products
In this notebook we will explore the power of graph products. We will use the following graph products:
* Cartesian product
* Strong product
* Tensor product

We will explore the use of different embeddings and the effect of different graph products on the embeddings.

In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from compare import compare_embeddings as compare
from counting import embed_graph_cycles as cycles
from weisfeiler_lehman import embed_graph_wl as wl
from product import get_all_products as products

We will start by using all possible non-isomorphic graphs with a maximum of 7 nodes. This gives us a total of 1253 graphs.

In [2]:
gs = nx.graph_atlas_g()

## Preprocessing
We will start by assembeling all the graphs and graph products in a structured manner to make it easier to analyze different embedding methods.

### Factor graphs
Using different factor graphs could change the embedding, and therefore the effect of the graph products. We will use the following factor graphs:
* Complete graph
* Path graph
* Star graph

Regarding the size of the factor graphs, we will use the following sizes:
* 3 nodes
* 5 nodes
* 7 nodes

In [3]:
factor_graphs = {
    'K3': nx.complete_graph(3),
    'K5': nx.complete_graph(5),
    'K7': nx.complete_graph(7),
    'P3': nx.path_graph(3),
    'P5': nx.path_graph(5),
    'P7': nx.path_graph(7),
    # star graph with 3 nodes is equivalent to path graph with 3 nodes
    'S5': nx.star_graph(5),
    'S7': nx.star_graph(7),
}

### Graph products
Now we will create the graph products of the factor graphs with all the graphs considered.

In [4]:
graph_products = pd.DataFrame(index=factor_graphs.keys(), columns=['Cartesian', 'Strong', 'Tensor'])
graph_products.index.name = 'Factor Graph'
graph_products.columns.name = 'Graph Product'

for factor in factor_graphs:
    FG = factor_graphs[factor]
    cartesian, strong, tensor = products(gs, FG)
    
    graph_products.loc[factor, 'Cartesian'] = cartesian
    graph_products.loc[factor, 'Strong'] = strong
    graph_products.loc[factor, 'Tensor'] = tensor


## Cycle counting
We will use the cycle counting method by preparing embedding each graph using a vector, where each entry contains the number of cycles of a certain length (the length is the index of the entry). We will then use different graph products and observe, how the cycle counts change.

### Embeddings
For all these graph products, as well as the initial graphs, we will create embeddings by counting cycles of different lengths. We will use an embedding size of 7*7=49, which is the maximum length of cycles in a graph product of two graphs with each 7 nodes.

In [5]:
cycle_embedded_graphs = cycles(gs, 49)
cycle_embedded_products = graph_products.map(lambda x: cycles(x, 49))

### Evaluation and Comparison
In this section, we will evaluate, how many graphs, that are isomorphic to each other, are mapped to the same embedding. We will also compare the embeddings of the different graph products.

In [6]:
cycle_results = compare(cycle_embedded_graphs)
cycle_product_results = cycle_embedded_products.map(lambda x: compare(x))

In [7]:
cycle_product_results

Graph Product,Cartesian,Strong,Tensor
Factor Graph,,,
K3,1429,12783,533
K5,1363,9751,350
K7,1362,9748,340
P3,1841,1242,8778
P5,2011,229,1234
P7,1882,170,683
S5,521,3193,8686
S7,521,1924,8686


## Weisfeiler-Lehman
Next will use the standard one dimensional Weisfeiler-Lehman isomorphisim test and perform the same evaluation and comparison as before.

### Embeddings
We will use 10 iterations of the Weisfeiler-Lehman algorithm to create the embeddings.

In [12]:
wl_embedded_graphs = wl(gs, 10)
wl_embedded_products = graph_products.map(lambda x: wl(x, 10))

### Evaluation and Comparison
In this section, we will evaluate the effectiveness of graph products to enhance the WL isomorphism test. 

In [13]:
wl_results = compare(wl_embedded_graphs)
wl_product_results = wl_embedded_products.map(lambda x: compare(x))

In [15]:
wl_results

26

In [14]:
wl_product_results

Graph Product,Cartesian,Strong,Tensor
Factor Graph,,,
K3,26,26,26
K5,26,26,26
K7,26,26,26
P3,26,26,26
P5,26,26,26
P7,26,26,26
S5,26,26,26
S7,26,26,26


### Conclusion and Discussion
As we can see from the table above, the graph products showed no effect on enhancing the WL isomorphism test.